In [1]:
# inicializacia credentials pre vertex ai a storage
import os
import google.auth
from google.cloud import aiplatform

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "prefab-shape-455916-i7-be937e48d8e1.json"

# na skusku endpoint pre stock-AAPL
aiplatform.init(
    project="4204132242382913536",
    location="europe-central2"
)

endpoint = aiplatform.Endpoint("4204132242382913536")

creds, project = google.auth.default()
print("Authenticated project:", project)
print("Credentials type:", type(creds))

Authenticated project: prefab-shape-455916-i7
Credentials type: <class 'google.oauth2.service_account.Credentials'>


In [8]:
# predickia modelu z endpointu na vertex ai
from google.cloud import aiplatform
from google.cloud import storage
import numpy as np
import yfinance as yf
import joblib
import io

def load_scaler_from_gcs(bucket_name, blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    scaler_bytes = blob.download_as_bytes()
    scaler = joblib.load(io.BytesIO(scaler_bytes))
    return scaler

def predict_next_days_vertexai(
        ticker, endpoint_id, project, location, days=5, time_steps=10, end_date='2025-04-05'
):
    # Initialize Vertex AI
    aiplatform.init(project=project, location=location)
    endpoint = aiplatform.Endpoint(endpoint_id)

    # Load scaler from GCS
    scaler = load_scaler_from_gcs("stock-prediction-zct", f"{ticker}_scaler.gz")

    # Download stock data
    start_date = '2020-04-01'
    data = yf.download(ticker, start=start_date, end=end_date)
    close_prices = data['Close'].values.reshape(-1, 1)
    close_prices_scaled = scaler.transform(close_prices)

    # Get the last time_steps data, reshape to (1, time_steps, 1)
    last_data = close_prices_scaled[-time_steps:]
    last_data = np.reshape(last_data, (1, time_steps, 1))

    future_predictions = []
    for _ in range(days):
        response = endpoint.predict(instances=last_data.tolist())
        pred = response.predictions[0][0]
        future_predictions.append(pred)

        last_data = np.append(last_data[:, 1:, :], np.array(pred).reshape(1, 1, 1), axis=1)

    print(future_predictions)
    future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))
    return future_predictions


preds = predict_next_days_vertexai(
    ticker="AAPL",
    endpoint_id="4204132242382913536",
    project="prefab-shape-455916-i7",
    location="europe-central2",
    days=10
)

#AAPL - 4204132242382913536
#NVDA - 8849595258015580160
#INT - 7367858204052553728

print(preds)

[*********************100%***********************]  1 of 1 completed


[0.74511826, 0.722068, 0.706082523, 0.693850577, 0.685101, 0.676949263, 0.671317875, 0.667188048, 0.664371848, 0.65790391]
[[207.68990335]
 [203.07358846]
 [199.87214972]
 [197.42243703]
 [195.67014432]
 [194.03758205]
 [192.90977437]
 [192.08268687]
 [191.51868169]
 [190.22333672]]
